<h1> Implementationen und Tests <h1>

Der erste Schritt der im Projekt implementiert wurde, ist die Leistungszahl (engl. figure of merit), die zu einem gegebenen Generatorvektor (engl. generating vector) einen Wert zurück gibt. Dieser sagt aus, wie gut besagter Vektor für die Erstellung eines Gitters geeignet ist. Hierbei sind geringere Werte besser. <br>
Im ersten Schritt wurden dafür die nötigen Pakete in Python implementiert und übersichtshalber die ersten kleinen Methoden definiert. <br>
<br>
c(n) definiert die Menge: $ C(N):= (-N/2,N/2] \cap \mathbb{Z}$ <br>
<br>
r1(h): Für $h \in \mathbb{Z}$ ist $r_1(h)=max(1,|h|)$


In [3]:
import numpy as np
import math
import time

def c(n):
    return np.array(range(int(-n/2),int(n/2+1)))

def r1(h):
    return max(1,abs(h))

<br>
Die eigentliche Leistungszahl ist nun folgendermaßen definiert: <br>
$$\begin{aligned} R_N(g)=-1 + \frac{1}{N} \sum \limits_{n=0}^{N-1} \prod \limits_{j=1}^s \phi(\frac{ng_j}{N}) \end{aligned}$$ 
mit
$$\begin{aligned} \phi(x)=\sum \limits_{h\in C(N)} \frac{1}{r_1(h)}exp(2\pi ihx) \end{aligned}$$
Zuerst wird $\phi (x)$ implementiert.

In [4]:
def phi(x,n):   
    result = 0
    for h in c(n):
        result = result + ((1/r1(h))*np.exp(2*math.pi*complex(0,1)*h*x))
    return result

<br>
Wenn wir jetzt jedoch ohne Weiteres $R_N(g)$ implementieren befinden wir uns bei einer Laufzeit im Bereich von $O(N^2)$. <br>
Da $\phi (ng/N)$ allerdings nur maximal $N$ verschiedene Werte annimmt - nämlich $\phi (ng/N)=\phi (k/N)$ für $k \in {0,1,...,N-1}$ - können wir die möglichen Werte schon im Voraus berechnen und in einem Array speichern. Statt $\phi (ng/N)$ zu berechnen können wir einfach den Eintrag an der Stelle $n  g (mod (N))$ aus dem Array nehmen.
Damit kommen wir in einen Bereich von $O(sN)$. <br>
<br>
Also erstellen wir zu erst eine Methode, die uns die Werte im vorraus berechnet und in ein Array schreibt, und im Anschluss die Berechnung der Leistungszahl mit Hilfe der Werte.

In [5]:
def precom(n): 
    precomvalues = [0]*n
    for i in range(0,n):
        precomvalues[i] = phi(i/n,n)
    return precomvalues

In [6]:
def fom_precom(g,n):
    result = 0
    precomvalues = precom(n)
    for i in np.array(range(0,n)):
        helper = 1 
        for x in g:
            helper = helper * precomvalues[(i*x)%n]
        result = result + helper
    merit =  result/n-1
    if merit.imag < 1e-14:
        return merit.real
    return "Error, figure of merit has a complex part"

In [8]:
def fom_precom_time(g,n): 
    start = time.process_time()
    fom = fom_precom(g,n)
    end = time.process_time()
    cputime = end - start
    print(fom,"   time: ", format(cputime))

<br>
Hierbei sei angemerkt, dass die Leistungszahl einen Imaginärteil mit sich führt, der aber fast immer in einem Bereich <1e-15 liegt, sodass er vernachlässigt werden kann. <br> <br>
Nun können wir dies an dem, vom Latnetbuilder erstellten, Generatorvektor $[1, 282, 197, 377, 233, 55, 73, 263, 408, 46]$ testen.

In [9]:
fom_precom_time([1, 282, 197, 377, 233, 55, 73, 263, 408, 46],1009)

436619963.19030744    time:  10.78125


<br>
Und auch der Generatorvektor aus der Matlab Implementation $[1,390,265,180,242,491,450,347,77,332]$ liefert einen ähnlichen Wert:

In [10]:
fom_precom_time([1, 390, 265, 180, 242, 491, 450, 347, 77, 332],1009)

436619963.311887    time:  10.453125


<br>
Allerdings wissen wir zur Zeit noch gar nicht, ob dies überhaupt ein passender Wert für die Leistungszahl ist. <br>
Aus dem Grund berechnen wir im nächsten Schritt den maximalen Wert, den die Leistungszahl überhaupt annehmen kann. Es gilt: <br>
$$\begin{aligned} R_N(g) \leq \frac{2^{s+1}(log N +1)^s}{N} \end{aligned}$$

In [12]:
def estimatemax(s,n):
   return 2**(s+1)*(math.log(n)+1)**s/n
estimatemax(10,1009)

1962858099.5554516